# Practical ResNet

### Imports

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

### Data

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 30
batch_size = 100
learning_rate = 0.001

transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

train_dataset = torchvision.datasets.CIFAR10(root='./data/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data/', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified


### Model

In [3]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
    
    def forward(self, x):
        identity = x.clone()
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

In [4]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        """
        Args:
            block (ResidualBlock): ResidualBlock class
            layers (list): [2, 2, 2]
            num_classes (int, optional): Defaults to 10.
        """
        super().__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0], stride=1)
        self.layer2 = self.make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self.make_layer(block, 64, layers[2], stride=2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
    
    def make_layer(self, block, out_channels, blocks, stride):
        downsample = None
        if self.in_channels != out_channels or stride != 1:
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        residual_blocks = []
        residual_blocks.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            residual_blocks.append(block(self.in_channels, out_channels))
        return nn.Sequential(*residual_blocks)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

### Training

In [5]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

In [8]:
from pytorch_training import train_model
train_model(num_epochs, model, criterion, optimizer, train_loader, test_loader, device, scheduler)

  0%|          | 0/30 [00:00<?, ?epochs/s]

Epoch 01/30:   0%|          | 0/600 [00:00<?, ?batches/s]

Epoch 02/30:   0%|          | 0/600 [00:00<?, ?batches/s]

Epoch 03/30:   0%|          | 0/600 [00:00<?, ?batches/s]

Epoch 04/30:   0%|          | 0/600 [00:00<?, ?batches/s]

...

Epoch 27/30:   0%|          | 0/600 [00:00<?, ?batches/s]

Epoch 28/30:   0%|          | 0/600 [00:00<?, ?batches/s]

Epoch 29/30:   0%|          | 0/600 [00:00<?, ?batches/s]

Epoch 30/30:   0%|          | 0/600 [00:00<?, ?batches/s]

{'train_loss': [1.2452103891372681,
  0.9544580736160279,
  0.8241738265752793,
  0.7311268132925034,
  0.6705218131542205,
  0.6167511087059975,
  0.5809827762246131,
  0.5464900395274163,
  0.5187937362194062,
  0.4951870012879372,
  0.476576937019825,
  0.4582934612035751,
  0.4394907240867615,
  0.4256089519262314,
  0.41548870053887366,
  0.40128834715485573,
  0.3884155831336975,
  0.38099647769331935,
  0.3264204211235046,
  0.313190875351429,
  0.3021153069883585,
  0.2999229906499386,
  0.29250863333046434,
  0.2865654668807983,
  0.27997230322659017,
  0.27375676229596135,
  0.27069105960428713,
  0.26667308785021304,
  0.261927552729845,
  0.2547426784783602],
 'train_accuracy': [0.54972,
  0.66248,
  0.71056,
  0.7459,
  0.76624,
  0.78592,
  0.7988,
  0.81142,
  0.82042,
  0.8303,
  0.83478,
  0.84026,
  0.84806,
  0.85284,
  0.85774,
  0.8613,
  0.86556,
  0.86674,
  0.88624,
  0.8916,
  0.89484,
  0.89434,
  0.89728,
  0.89876,
  0.90238,
  0.90314,
  0.90528,
  0.9059,
